In [5]:
import time, os
import pandas as pd
from tqdm import tqdm
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert

Exception ignored in: <function tqdm.__del__ at 0x7fd79832e430>
Traceback (most recent call last):
  File "/Users/shimsomin/opt/anaconda3/envs/timeline/lib/python3.9/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/Users/shimsomin/opt/anaconda3/envs/timeline/lib/python3.9/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


- 7월: ***19,240건***
- 8월: ***20,007건***
- 9월(30): ***15,993건***
- 10월: ***18,726건***
- 11월(30): ***20,991건***
- 12월: ***17,010건***

In [6]:
jul = 19240//5
aug = 20007//5
sept = 15993//5
oct = 18726//5
nov = 20991//5
dec = 17010//5

In [52]:
# 드라이버 지정
current = '/Users/shimsomin/Documents/GitHub/COVID19-Tourism'
chrome_path = current + '/driver/chromedriver'

options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
options.add_argument('headless') # 창 안 띄움 -> 더 빠름
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # Headless 탐지 막기위해 User-Agent 값을 바꿔줌.

options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

#Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

query = "제주 여행"
start_date = "2020-08-01"
end_date = "2020-08-31"
max_link_cnt = aug
crawling_month = '08'

naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate={}&endDate={}&keyword={}".format(start_date, end_date, query)


In [53]:
driver.get(url)
soup = BeautifulSoup(url)

# 1차 크롤링 (url, 제목) 시작
link_cnt = 0
btn_cnt = 1
while link_cnt <= max_link_cnt:
    # 한 페이지 안의 총 기사 개수
    t_len = len(driver.find_elements_by_class_name("list_search_post"))

    for i in range(1, t_len+1):
        a = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner".format(i))

        # url 수집
        url = a.get_attribute('href')

        # 제목 수집
        title = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span".format(i)).text
        
        naver_df.loc[link_cnt] = ['nan', '네이버', query, title, 'nan', url, 'nan']

        link_cnt += 1
    print(link_cnt)
    
    btn_cnt += 1
    # 다음 버튼 누르기
    if btn_cnt % 10 == 1:
        if btn_cnt > 11:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a.button_next")
            btn.send_keys(Keys.ENTER)
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a")
            btn.send_keys(Keys.ENTER)
        driver.implicitly_wait(3)
    else:
        if btn_cnt > 11:
            if (btn_cnt % 10) == 0:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child(11) > a")
            else:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format((btn_cnt%10)+1))
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format(btn_cnt))

        btn.send_keys(Keys.ENTER)
        driver.implicitly_wait(3)


/Users/shimsomin/opt/anaconda3/envs/timeline/lib/python3.9/site-packages/bs4/__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 197 of the file /Users/shimsomin/opt/anaconda3/envs/timeline/lib/python3.9/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


0
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
168
175
182
189
196
203
210
217
224
231
238
245
252
259
266
273
280
287
294
301
308
315
322
329
336
343
350
357
364
371
378
385
392
399
406
413
420
427
434
441
448
455
462
469
476
483
490
497
504
511
518
525
532
539
546
553
560
567
574
581
588
595
602
609
616
623
630
637
644
651
658
665
672
679
686
693
700
707
714
721
728
735
742
749
756
763
770
777
784
791
798
805
812
819
826
833
840
847
854
861
868
875
882
889
896
903
910
917
924
931
938
945
952
959
966
973
980
987
994
1001
1008
1015
1022
1029
1036
1043
1050
1057
1064
1071
1078
1085
1092
1099
1106
1113
1120
1127
1134
1141
1148
1155
1162
1169
1176
1183
1190
1197
1204
1211
1218
1225
1232
1239
1246
1253
1260
1267
1274
1281
1288
1295
1302
1309
1316
1323
1330
1337
1344
1351
1358


In [ ]:
# 2차 크롤링 (블로그 제목, 포스팅 날짜, 글쓴이) 시작
for i in tqdm(range(len(naver_df))):
    driver.get(naver_df.loc[i]['url'])
    driver.switch_to.frame('mainFrame')
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements_by_css_selector(overlays)

    content_list = ""

    for content in contents:
        content_list = content_list + content.text

    # 글쓴이 수집
    author = driver.find_element_by_xpath("""//*[@id="nickNameArea"]""").text
    
    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df.loc[i][['날짜', '내용', '글쓴이']] = [date, content_list, author]

In [47]:
naver_df.tail()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
3845,2020. 7. 4. 12:02,네이버,제주 여행,"[제주여행] 1일차. 동문시장 먹거리 탐방 - 갈치회, 고등어회, 딱새우회, 한치회...","안녕! 고소밍입니다 :)\n5월부터 일태기, 회태기가 왔어요. 지난 포스팅에서도 언...",https://blog.naver.com/aishaniverse/222020783560,고소밍
3846,2020. 7. 5. 22:53,네이버,제주 여행,[제주여행] 협재해수욕장의 오션뷰 스타벅스 '스타벅스 제주협재점',"아이스 아메리카노 그란데 4,600원\n자몽셔벗블렌디드 (무료쿠폰)제주도에 스타벅스...",https://blog.naver.com/ekdud05310/222022055803,여행가고싶다
3847,2020. 7. 6. 0:00,네이버,제주 여행,7월 제주 여행 애월 가볼만한 곳,,https://blog.naver.com/risingyou21/222022124400,제주앓이
3848,2020. 7. 3. 16:52,네이버,제주 여행,[국내여행 ] 제주여행 2일차 ( 사계전복 / 카멜리아힐 / 오설록 / 순천미향...,동생은 김포에서 일때문에 따로 오기로 해서 \n이틀차에 만나기로 했다 \n엄마가 용...,https://blog.naver.com/wo_n94/222020113618,wony
3849,2020. 7. 6. 21:14,네이버,제주 여행,두 바보의 제주 여행 이야기,# 이상한 셈법\n니가 운전하니까 회계는 내가 해 주지!! 흐흐흐..\n친구의 자진...,https://blog.naver.com/philjackie/222023036258,재키


In [ ]:
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

In [ ]:
# 에러 발생시 비프음 알림 울리게 하기
try:
    print()
except:
    beepsound()

In [ ]:
crawling_month

In [49]:
save_dir = current+'/data'
naver_df.to_csv(save_dir + '/naver_2020_{}.csv'.format(crawling_month), index=False)